In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

In [2]:
sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), "../../DermSynth3D_private"))
)
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../skin3d")))

from dermsynth3d.datasets.datasets import (
    Fitz17KAnnotations,  # ?
    Background2d,
)
from dermsynth3d.datasets.synth_dataset import (
    SynthesizeDataset,
)
from dermsynth3d.utils.filestruct import custom_directories
from dermsynth3d.tools.generate2d import Generate2DHelper
from dermsynth3d.utils.colorconstancy import shade_of_gray_cc

In [3]:
from skin3d.bodytex import BodyTexDataset

In [4]:
# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

In [5]:
my_folders = custom_directories("jer")
# my_folders = custom_directories("arezou")
# my_folders = custom_directories("ashish")
print(my_folders)

/mnt/d/data/archive/House_Room_Dataset/Bedroom/
/mnt/d/data/3dbodytex-1.1-highres/3dbodytex-1.1-highres/
/mnt/d/data/3dbodytex-1.1-highres/3DBodyTex_nonskinfaces/3DBodyTex_nonskinfaces
/mnt/d/data/bodytex/bodytex_anatomy/bodytex_anatomy_labels
/mnt/d/data/fitzpatrick17k/data/finalfitz17k
/mnt/d/data/fitzpatrick17k/annotations/annotations-20220429T234131Z-001/annotations


In [6]:
# File path of the bodytex CSV.
bodytex_csv = "../skin3d/data/3dbodytex-1.1-highres/bodytex.csv"
bodytex_df = pd.read_csv(bodytex_csv, converters={"scan_id": lambda x: str(x)})
bodytex = BodyTexDataset(
    df=bodytex_df,
    dir_textures=my_folders.bodytex_highres(),
    dir_annotate="../skin3d/data/3dbodytex-1.1-highres/annotations/",
)

In [7]:
# Extension of the texture image. Use the extension from `paste_lesions.ipynb`
TEXTURE_EXTENSION = "tester"
# True to use the blended lesions, False to use the pasted lesions.
is_blend = True

# Create a 2D dataset based on rendered views of the 3d model.
# Specify the directories where to save the images and masks.
# You'll want to change this directory for your experiments.
# This will create a new directory if it does not exist.
SYNTH_DIR_NAME = "tester"
dir_synth_data = os.path.join("/mnt/d/data/synth/", SYNTH_DIR_NAME)
synth_ds = SynthesizeDataset(dir_synth_data)

In [8]:
# background_ds = Background2d(
#     dir_images=my_folders.backgrounds(),
#     )

dir_background_images = "/mnt/d/data/backgrounds/IndoorScene1/IndoorScene1"
background_ds = Background2d(
    dir_images=dir_background_images,
    image_filenames=None,
)

In [9]:
print(my_folders.new_textures())
subject_ids = sorted(os.listdir(my_folders.new_textures()))
len(subject_ids)

/mnt/d/data/3dbodytex-1.1-highres/3DBodyTex_nonskinfaces/3DBodyTex_nonskinfaces


129

In [ ]:
subject_ids = ["221-m-u"]
# subject_ids = ['370-m-a', '371-m-scape015', '382-m-a', '386-m-a', '395-m-scape070']
for subject_id in subject_ids:
    print("===== WORKING ON {}".format(subject_id))
    scan_id = subject_id[:3]
    nevi_exists = os.path.exists(bodytex.annotation_filepath(scan_id))
    if not nevi_exists:
        # raise ValueError("Missing bodytex annotations.")
        print("missing bodytex annotations: {}".format(subject_id))
        continue

    mesh_filename = os.path.join(
        my_folders.bodytex_highres(), subject_id, "model_highres_0_normalized.obj"
    )

    # This will load the meshes and necessary files.
    # Assumes you already have the blended textured for this mesh.
    # Or set `is_blended=False` to use the pasted lesions.
    # print("===== Loading mesh")
    gen2d = Generate2DHelper(
        mesh_filename=mesh_filename,
        dir_blended_textures=my_folders.new_textures(),
        dir_anatomy=my_folders.anatomy(),
        fitz_ds=None,  # fitz_ds,
        background_ds=background_ds,
        device=device,
        debug=True,
        bodytex=bodytex,
        blended_file_ext=TEXTURE_EXTENSION,
        config=None,
        is_blended=is_blend,
    )

    n_imgs = 10  # Total number of 2D images to save to disk.
    img_count = 0  # Counts the number of images saved to disk.
    gen2d.debug = True  # Turn on/off the printing of debug messages.
    while img_count < n_imgs:  # While loop to keep rendering until `n_imgs` are saved.
        view_size = (512, 512)  # Image size of rendered view.
        success = gen2d.randomize_parameters(
            view_size=view_size,
            config=None,
            sample_mode="sample_surface",
            # surface_offset_bounds = (0.1, 1.0),
            background_blur_radius_bounds=(0, 0),
        )
        if (
            not success
        ):  # Checks if the camera/lighting placement works for the random params.
            print("***Camera and lighting placement not successful. Skipping")
            continue

        # Option to paste the lesion.
        paste_img, target = gen2d.render_image_and_target(
            paste_lesion=False,
            min_fraction_lesion=0.0,
        )
        if paste_img is None:  # Checks if enough skin is visible.
            print("***Not enough skin, lesion, or unable to paste lesion. Skipping.")
            continue

        target_name = synth_ds.generate_target_name()
        # print(target_name)
        # Save image and masks to disk.
        synth_ds.save_image(target_name, (paste_img * 255).astype(np.uint8))
        synth_ds.save_target(target_name, target)

        # Keep track of the parameters used to generate the image.
        params = {
            "file_id": target_name,
        }
        params.update(gen2d.get_params())
        synth_ds.update_params(params)
        img_count += 1  # Increment counter.
        print(img_count)

    # Save the params to disk (make sure you do this outside the while loop)
    print("===== Saving parameters ...")
    synth_ds.save_params()